In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
pd.set_option('display.max_columns', 500)
pd.set_option("display.max_colwidth", -1)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# Introduction
* In this notebook, we'll dive into the football events dataset to look at various stats that are hidden and try to build an XG (expected goal) model to predict how many goals a player should have scored.
* The dataset was created by extracting information from the text of each event and analysing for keywords to obtain various event types such as fouls, off-sides, shot locations, attempt on goal etc, which can be found in the dict file accompanied with this dataset.
* The dataset provides a granular view of 9,074 games, totaling 941,009 events from the biggest 5 European football leagues: England, Spain, Germany, Italy, France from 2011/2012 season to 2016/2017 season as of 25.01.2017.
* We'll look at some basic visualizations and perform some exploratory analysis to answer questions such as 'which league is the best' or top goal scorer, etc.


### Ambiguity
* First lets address the ambiguity about the matches that actually have detailed events about them as the OP clearly states that about 90% of the data have detailed event records. We need to determine the number of matches recorded in each league.

In [ ]:
events=pd.read_csv('../input/football-events/events.csv')
match=pd.read_csv('../input/football-events/ginf.csv')
print(f'number of unique matches in events dataset: {np.size(events.id_odsp.unique())}',
      f'number of unique matches in match dataset: {np.size(match.id_odsp.unique())}',sep='\n')

The 'match' dataset has more number of matches than the 'events' dataset, hence there are some matches for which there are no advanced statisitc

In [ ]:
pd.crosstab(match.country,match.season)

* As all the leagues except Germany have 20 participating teams, the total number of matches played in each league is 380 matches per season and 308 matches per season in Germany for the 18 participating teams.
* Lets look at the distribution of matches in each league that actually have detailed stats about them
*  As a note, the 2011/2012 season is recorded as 2012 

In [ ]:
pd.crosstab(match.country[match.adv_stats==True],match.season[match.adv_stats==True])

In [ ]:
#merging the two tables to obtain only those matches which have advanced statistics by joining on the 'id_odsp' variable
#which is an unique match indentifier key present in both the tables
merged_tables=pd.merge(events,match,on='id_odsp',how='inner')
sns.distplot(merged_tables.groupby('id_odsp').count().id_event,kde=False)
plt.show()

For most of the matches, there are 100 events per match recorded in the dataset.

### visualizations
As most of the variables are categorical variables, we can use catplot from seasborn to visualize the correlation between the different variables. Plenty of visualizations can be made as there are more than 30 categories in the dataset, but lets pick out 'assist_method' and 'bodypart' variables and plot them to get some insights.

In [ ]:
sns.catplot(data=events,x='assist_method',y='bodypart',kind='violin')
plt.xticks(ticks=range(5),labels=['None','pass','cross','headed','through ball'])
plt.yticks(ticks=range(4),labels=['0','right foot','left foot','head'])
plt.show()

As we can see, majority of the crosses attempted by the players were headed onto the goal which is pretty obvious but altogehter fun to visualize. 
Its said that only 10% of the population is left handed. Here we can see that any event that resulted in interaction with the ball, most of it was with the right foot. Hence, as seen in the world, our dataset has a lot of right footed players. We'll see more visualizations along the way.

### Statistical comparision of the leagues
Lets statistically compare the 5 leagues by comparing the values of 'event_type' and using chi-square test on the obtained crosstab. As the number of matches for each league is different, we'll have to divide the crosstab by the number of matches in that league to obtain stats for occurence of an event_type per match. 
It is important to note that then number of matches in the 'events' dataset and match dataset are different, hence we will have to look at the matches included only in the 'events' dataset. This is due to the fact that all the matches recorded in the 'match' dataset do not have advanced statistics ('adv_stats' column in the 'match' dataset) about them.

In [ ]:
country_count=pd.crosstab(merged_tables.country,merged_tables.event_type)
country_count

In [ ]:
# using unique() method to determine all the matches that are present in the merged_tables and using isin() to return
# a boolean array to be applied on match dataset to obtain number of matches
unique_matches=match[match.id_odsp.isin(merged_tables.id_odsp.unique())]
unique_matches.country.value_counts()

In [ ]:
key=dict(unique_matches.country.value_counts())
for i in key:
    country_count.loc[i]=country_count.loc[i]/key[i]
country_count

At a glance, there seems to be no difference between the leagues in terms of 'event_type' such as attemps, corners, fouls, substitutions etc, and is a true fact which will be confirmed by the chi-square stats. Although at an average, the difference between england and other leagues in terms of fouls awarded per match is 5 fouls. Maybe England ref's are a bit linient.

In [ ]:
from scipy.stats import chi2_contingency
chi2_contingency(country_count)[1]

The given stat is the p-value, and a high p-value means that the null hypothesis that the leagues are independent cannot be rejected. Hence, the leagues are all the same i.e. with the information from the dataset. Leagues are compared by the style of play of football, the intensity of pressing the ball, the speed of play etc, which cannot be inferred from the given information.

### Trivia
* The dataset has over 9000 unique matches ranging over 6 seasons, so we should be able to answer some questions such as who has been the top scorer, assister and much more can be inferred.
* We can look at how accurate the bookies were in predicting the outcome of each game.


In [ ]:
# extracting top 10 players who scored more than 80 goals in the given timeframe so as to create a new table consisting of
# only their events
names=pd.crosstab(merged_tables.player,merged_tables.is_goal).drop(0,axis=1).sort_values(by=1,ascending=False).head(10).index
stats_table=merged_tables[merged_tables.player.isin(names)]

#top 10 goal scorers
top10=pd.crosstab(stats_table.player,stats_table.is_goal).drop(0,axis=1)
top10.columns=['goals']

#total number of attempts on goal
attempts=pd.crosstab(stats_table[stats_table.event_type==1].player,stats_table[stats_table.event_type==1].event_type)
attempts.columns=['attempts']

#attempts by body part
#where atmps is 'attempts'
body_part_attempts=pd.crosstab(stats_table.player,stats_table.bodypart)
body_part_attempts.columns=['right atmps','left atmps','header atmps']

#bodypart attempts that lead to a goal
#where fg is 'foot goal'
successful_attempt=pd.crosstab(stats_table[stats_table.is_goal==1].player,stats_table[stats_table.is_goal==1].bodypart)
successful_attempt.columns=['right fg','left fg','header g']

#conversion_rates
conversion_rate=pd.DataFrame(top10.goals/attempts.attempts)
conversion_rate.columns=['conversion_rate']

#conversion_rates by body parts. This is a good estimator of the preferred foot of a player. Player attempts can also be used
#to determine the preferred foot as a player attempts more shots with their preferred foot
#where c/r is 'conversion rate'
body_part_conversion=pd.DataFrame(columns=['right c/r','left c/r','header c/r'])
body_part_conversion['right c/r']=successful_attempt['right fg']/body_part_attempts['right atmps']
body_part_conversion['left c/r']=successful_attempt['left fg']/body_part_attempts['left atmps']
body_part_conversion['header c/r']=successful_attempt['header g']/body_part_attempts['header atmps']

#concatinating all the tables to obatin insights on the 10 choosen players
trivia=pd.concat([top10,attempts,conversion_rate,body_part_attempts,successful_attempt,body_part_conversion],axis=1).sort_values(by='goals',ascending=False)
trivia

In [ ]:
sns.set_style('darkgrid')
sns.relplot(data=trivia,x='right atmps',y='left atmps',hue=trivia.index,size='header atmps',sizes=list(range(50,550,50)) ,
            palette='bright',aspect=1.5,marker='D')
plt.show()

The graph is able to use 3 different variables that are, the right footed attempts, left footed attempts and the size of the markers as the number of headed attempts. This plot shows us the preferred foot of the players. 

In [ ]:
%matplotlib inline
placeholder=trivia.attempts/trivia.goals
sns.set_style('darkgrid')
fig, ax = plt.subplots(figsize=(12,6))
sns.set_color_codes(palette='deep')
sns.barplot(data=trivia,x='attempts',y=trivia.index,ax=ax,palette='winter',label='attempts')
sns.barplot(data=trivia,x='goals',y=trivia.index,ax=ax,palette='copper',label='goals')
for i in range(0,10):
    plt.text(0.75*trivia.attempts[i],i,'@'+str(round(placeholder[i],ndigits=2))+' '+'shots/goal')
plt.xlabel('goals and attempts')
plt.legend()
plt.show()
#The top 10 player sorted according to the goals scored.

* from the conversion rates we can see that it takes Messi more than 4 (1/0.224289) attempts on goal to score whereas, it takes Ronaldo 6 attempts to score. Of the 10, Diego Costa has the best conversion rate meaning he is deadliest attacker of the 10
* the distribution of attempts among the body parts variable is a good indicator of the players preferred foot when in front of goal. 
* when examining the conversion rates according to the body part, Diego Costa,Pierr-Eemerick Aubameyang, Robert lewandowski and Zlatan Ibrahimovic have almost identical conversion rates with either of their foot meaning that they are equally as lethal in front of goal with either foot, mark of a Great Striker.	 

In [ ]:
#To calculate the number of times each player was involved in an attempt on goal, i.e. if he shot at goal or his pass was a key pass
player1=pd.crosstab(merged_tables[merged_tables.event_type==1].player,merged_tables[merged_tables.event_type==1].event_type).sort_values(by=1,ascending=False)
player2=pd.crosstab(merged_tables[merged_tables.event_type==1].player2,merged_tables[merged_tables.event_type==1].event_type).sort_values(by=1,ascending=False)
involvements=player1.join(player2,lsuffix='_player1')
#to get the total number of involvements
involvements['total_involvements']=involvements['1_player1']+involvements['1']
involvements=involvements.sort_values(by='total_involvements',ascending=False)
involvements.columns=['attempt on goal','key pass','total_involvements']
involvements.head(20)

In [ ]:
#lets determine the number of matches played by these players to get an interesting stats, the number of involvements per match
#to determine stats of around 40 players.
players=involvements.head(40).index
appearances=[]
for i in players:
    appearances.append(len(events[(events.player==i)|(events.player2==i)].groupby(by='id_odsp').count()))
involvements=involvements.head(40)
involvements['appearances']=appearances
involvements['involvements/match']=involvements.total_involvements/involvements.appearances
sns.barplot(data=involvements.sort_values(by='involvements/match',ascending=False).head(15),
            x='involvements/match',y=involvements.sort_values(by='involvements/match',ascending=False).head(15).index
           ,palette='plasma')
plt.show()

* Above graph gives a good estimate on the players with more than 4 involvements that resulted in an attempt on goal. They can be called the Top creaters in Europe.
Lets look at the predictions made by the bookies.

In [ ]:
#first defining a new column which records the actual outcome of each match
outcome=[]
for i in range(len(match)):
    if match.fthg.iloc[i]>match.ftag.iloc[i]:
        outcome.append('home win')
    elif match.fthg.iloc[i]<match.ftag.iloc[i]:
        outcome.append('away win')
    else: outcome.append('draw')

#concatenating the new column with the predicitons made by the bookies
bookie_accuracy=pd.concat([match[['odd_h','odd_d','odd_a']],pd.Series(outcome)],axis=1)

# defining a column which records the predictions made by each bookies by comparing the odds values
predictions=[]
for i in range(len(match)):
    a=[match.odd_h.iloc[i],match.odd_d.iloc[i],match.odd_a.iloc[i]]
    if min(a)== match.odd_h.iloc[i]:
        predictions.append('home win')
    elif min(a)== match.odd_d.iloc[i]:
        predictions.append('draw')
    elif min(a)== match.odd_a.iloc[i]:
        predictions.append('away win')
        
bookie_accuracy=pd.concat([bookie_accuracy,pd.Series(predictions)],axis=1)
bookie_accuracy.columns=['odd_h','odd_d','odd_a','outcome','prediction']

import sklearn.metrics as sk
print('        h_win  draw  a_win',
      f'home win {sk.confusion_matrix(bookie_accuracy.outcome,bookie_accuracy.prediction)[0]}',
      f'     draw{sk.confusion_matrix(bookie_accuracy.outcome,bookie_accuracy.prediction)[1]}',
      f'away win {sk.confusion_matrix(bookie_accuracy.outcome,bookie_accuracy.prediction)[2]}',sep='\n')


The bookies were not that aaccurate in predicting the matches. They predicted around 50% of the home matches correctly, 2.8% of the draws and 84% of the away wins were predicted correctly. In summary, the bookies predicted 53.35% of the matches correctly. (actual outcomes are the rows and predictions are the columns)

# Xg Model
##### expected goals
* Expected goals (xG) is a predictive model used to assess every goal-scoring chance, and the likelihood of scoring. A xG model computes for each chance the **probability** to score based on what we know about it (event-based variables). The higher the xG - with 1 being the maximum, as all probabilities range between 0 and 1  - the higher the probability of scoring. In practice, that means if a chance has 0.2xG, it should be scored 20 per cent of the time. If it has 0.99xG, it should be converted 99 per cent of the time and so on.
* But we will keep our model simple by using the available information and applying classification models so as to obtain a 'goal' or 'no goal' as the predicitons.
* According to the official Bundesliga website, (https://www.bundesliga.com/en/bundesliga/news/expected-goals-xg-model-what-is-it-and-why-is-it-useful-sportec-solutions-3177), the important points to look at for an XG model are:
    * Distance to the goal:- this information can be gathered from the 'location' variable in the events dataset.
    * Angle to the goal:- this information too can be gathered from the 'location' variable. 
    * Did the player strike it with his feet or was it a header?:- information about the body part used to attack the ball is given in the 'body_part' variable in the events dataset.
    * In what passage of play did it happen? (e.g. open play, direct free-kick, corner kick, counter-attack):- information about this can be gathered from the 'situation' variable in the events dataset.
    * Has the player just beaten an opponent?:- this is not available in the data set, but we can include additional data such as 'shot_place' which gives us information about the placement of the shot w.r.t the goal.
* The events dataset records all the events ranging from a foul to a subsitution, hence we should filter out the data by using only those events whihc are an attempt on goal (event_type==1) and avoid all own goals (event_type2!=15).


In [ ]:
xg_stats=events[['is_goal','location','assist_method','bodypart','situation','shot_place']][(events['event_type']==1)&(events['event_type2']!=15)]
xg_stats.describe()

In [ ]:
#identifying all the NaN's in the shot_place variable so as to remove them or replace the NaN's with some specific value
events.iloc[xg_stats[xg_stats.shot_place.isnull()].index].text.head()

Upon inspecting the text for which the shot_place is a NaN, we see that most of the text say that thee shot_place was at the 'Center of the goal'. Just replacing all the NaN's with the value 5 ('Center of the goal' category for 'shot_place') would be hasty, but we cannot examine each text manually. Therefore, by cycling through each text and comparing it to the categories of the shot_place variable, we'll be able to remove the NaN's.

In [ ]:
#to extract the text for all NaN's 
to_use=events[(events.event_type==1)&(events.shot_place.isnull())&(events.event_type2!=15)].text
key={1:'bit too high',2:'blocked',3:'bottom left corner',4:'bottom right corner',5:'centre of the goal',6:'high and wide',7:'Hits the bar',
     8:'misses to the left',9:'misses to the right',10:'too high',11:'top centre of the goal',12:'top left corner',13:'top right corner'}
for i in to_use.index:
    for k in key:
        if key[k] in to_use.loc[i].casefold():
            xg_stats.shot_place.loc[i]=k
        else:pass

In [ ]:
xg_stats.describe()

In [ ]:
#as there are some NaN's left over, about 11 of them, we'll just remove those entries
xg_stats=xg_stats.drop(xg_stats[xg_stats.shot_place.isnull()].index)
xg_stats.describe()

### correlation and multicollinearity 
* As the variables are all categorical, a simple correlation matrix using pearson's correlation is not feasible to gauge the relationships between the variables
* Hence we can use Cramers correlation coefficient to determine the relationship between the variables and check for collinearity by visualizing the correlation matrix using a heatmap 

In [ ]:
from scipy.stats import chi2_contingency
def cramer(var1,var2):
    crosstab=pd.crosstab(var1,var2)
    chi2=chi2_contingency(crosstab)[0]
    minn=min(crosstab.shape)-1
    return np.sqrt(chi2/(minn*np.sum(np.sum(crosstab))))


columns=[]
for i in xg_stats.columns:
    rows=[]
    for j in xg_stats.columns:
        value=cramer(xg_stats[i],xg_stats[j])
        rows.append(value)
    columns.append(rows)

    
corr_matrix=pd.DataFrame(columns,columns=xg_stats.columns,index=xg_stats.columns)
fig,ax=plt.subplots(1,2,figsize=(10,4))
sns.heatmap(corr_matrix,ax=ax[0])
sns.violinplot(data=xg_stats,x='is_goal',y='location',ax=ax[1])
plt.yticks(ticks=range(1,20))
plt.show()  


In [ ]:
#importing all the required modules and classifier models
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from sklearn.tree import DecisionTreeClassifier

In [ ]:
#as all the data are categorical variables, we need to convert them into dummy variables
for_season=pd.merge(events,match,on='id_odsp',how='inner')
for_season=for_season.iloc[xg_stats.index]
xg_stats_final=pd.concat([pd.get_dummies(xg_stats.location),
               pd.get_dummies(xg_stats.shot_place,prefix='shot_place'),
               pd.get_dummies(xg_stats.situation,prefix='sit'),
               pd.get_dummies(xg_stats.assist_method,prefix='ass'),               
               xg_stats.is_goal],axis=1)
xg_stats_final['season']=for_season.season


### Cross-validation and model fitting
* Cross-validation is the method to determine which model to use by comparing the accuracies of the different models without worrying about diving the dataset into a training set, testing set or validation set. It is done by the cross validation method by using k-1 data points from the given information and testing using the remaining information and repeated using different combinations during each iteration. Hence we can deterimine, in this case, if Logistic regression is a better model than Decision Tree Classifier
* First we will have to divide the dataset into a training set and a validation set. Let the 2016 season be the datapoints which we leave out from the training set so as to use it to predict and evaluate the performance of the model

In [ ]:
train=xg_stats_final[xg_stats_final.season!=2016]

In [ ]:
logreg=LogisticRegression(solver='liblinear',C=0.025)
#Here, C is used to tune the Regularization of the model.It is the inverse of regularization strength.
#Like in support vector machines, smaller values specify stronger regularization.
y_pred=cross_val_predict(logreg,train.drop(['is_goal','season'],axis=1),train.is_goal,cv=5)
precision_score(train.is_goal,y_pred)

In [ ]:
tree_model=DecisionTreeClassifier()
y_pred=cross_val_predict(tree_model,train.drop(['is_goal','season'],axis=1),train.is_goal,cv=5)
precision_score(train.is_goal,y_pred)

In [ ]:
#choosing Logistic Regression and using train_test_split to randomly sample test and train data from train dataset
x_train,x_test,y_train,y_test=train_test_split(train.drop(['is_goal','season'],axis=1),train.is_goal,test_size=0.2)
logreg.fit(x_train,y_train)
y_reg_pred=logreg.predict(x_test)
precision_score(y_test,y_reg_pred)

In [ ]:
# testing the model on the 2016 data of specific players 
lets_test=xg_stats_final.loc[for_season[(for_season.season==2016)&(for_season.player=='jamie vardy')&(for_season.event_type==1)].index]
y_pred=logreg.predict(lets_test.drop(['is_goal','season'],axis=1))
print(f'precision score = {precision_score(lets_test.is_goal,y_pred)}',f'predicted counts = {np.unique(y_pred,return_counts=True)}',sep='\n')
print('   0    1',f'0 {confusion_matrix(lets_test.is_goal,y_pred)[0]}',f'1 {confusion_matrix(lets_test.is_goal,y_pred)[1]}',sep='\n')

According to our model, Vardy should have scored 11 goals. But he actually scored 24 goals that year with an XG of 23.08. Here our model's precision score seems to show an accuracy of 90.9% but still could not predict the number of goals scored correctly. This is due to an **unbalanced** dataset.
* Unbalanced here means the number of 'goal' events and 'no-goal; events are substantially different. As seen ahead, the number of 'goal'(1) events account for only 10% of the dataset whereas the remaining are all 'no-goal'(0) events. Hence our model predicts almost all no-goal events correctly, but the events that matter i.e. the goals, are also predicted as no-goals, hence the high accuracy score but low relaibilty.
* Take for example fraud detection. If 99/100 times a transaction is not a fraud, and a model that predicts it, has a 99% accuracy, it will predict the harmless transactions accurately, but when we need it the most , i.e. when there is a fraudulent transaction, it may fail to predict it. Hence, when the weight of the predicted outcome is different or the dataset is imbalanced, accuracy will not be a good measure of effectiveness of the model.
* To counter this, lets reduce the number of no-goal events and see the performance of the model, by reducing the amount of no-goal events to avout 60%-70% of the dataset.

In [ ]:
train.is_goal.value_counts()

In [ ]:
new_train=pd.concat([train[train.is_goal==0].sample(n=79144),train[train.is_goal==1]])
x_train,x_test,y_train,y_test=train_test_split(new_train.drop(['is_goal','season'],axis=1),new_train.is_goal,test_size=0.2)
logreg1=LogisticRegression(solver='liblinear',C=0.025)
logreg1.fit(x_train,y_train)
lets_test=xg_stats_final.loc[for_season[(for_season.season==2016)&(for_season.player=='jamie vardy')&(for_season.event_type==1)].index]
y_test=logreg1.predict(lets_test.drop(['is_goal','season'],axis=1))
print(f'precision score = {precision_score(lets_test.is_goal,y_test)}',f'predicted counts = {np.unique(y_test,return_counts=True)}',sep='\n')
print('   0    1',f'0 {confusion_matrix(lets_test.is_goal,y_test)[0]}',f'1 {confusion_matrix(lets_test.is_goal,y_test)[1]}',sep='\n')

Our model predicted that, Jamie Vardy should have scored 22 goals in the 2016 season. According to infogol  (https://www.infogol.net/en/player/jamie-vardy/6147), Vardy scored 24  goals, outperforming his 23.08 XG by a goal.
There are different ways to interpret this model, one of them is: {*0 (no-goal) is considered as negative and 1 (goal) is considered as positive to think about false positives and negatives intuitively*}
* The 8 goal events (false positives) that were actually scored by vardy were deemed as 0 (no-goal) by our model. Hence in real life, Vardy defied the model by scoring those 8 goals, meaning those 8 goals were so difficult to score, there were interpretted as 'not a goal'(0).
* Vardy scored 16 goals when given chances that should have resulted in a goal. The remianing 6 goals (false positives) that were predicted, are those chances that our model predicted should as a goal but in real life were not scored by Vardy. By those predictions, in real life, Vardy scuffed those chances.
* So, according to our model, Jamie Vardy outperformed by 2 goals by scoring 24, ending the season with an XG of 22.

In [ ]:
lets_test=xg_stats_final.loc[for_season[(for_season.season==2016)&(for_season.player=='harry kane')&(for_season.event_type==1)].index]
y_test=logreg1.predict(lets_test.drop(['is_goal','season'],axis=1))
print(f'precision score = {precision_score(lets_test.is_goal,y_test)}',f'predicted counts = {np.unique(y_test,return_counts=True)}',sep='\n')
print('   0    1',f'0 {confusion_matrix(lets_test.is_goal,y_test)[0]}',f'1 {confusion_matrix(lets_test.is_goal,y_test)[1]}',sep='\n')

Our model predicted that Harry kane should have scored 24 goals. According to infogol (https://www.infogol.net/en/player/harry-kane/4959), he scored 25 goals that season ending with as XG of 22.73 meaning he outperformed by 2.27 goals. We can interpret the predicitons of our model as: {*0 (no-goal) is considered as negative and 1 (goal) is considered as positive to think about false positives and negatives intuitively*}
* The 8 goal events (false positives) that were actually scored by Kane were deemed as 0 (no-goal) by our model. Hence in real life, Vardy defied the model by scoring those 8 goals, meaning those 8 goals were so difficult to score, they were interpretted as 'not a goal'(0).
* Kane scored 17 goals when given chances that should have resulted in a goal. The remianing 7 goals (false positives) that were predicted, are those chances that our model predicted should as a goal but in real life were not scored by Kane. By those predictions, in real life, Kane scuffed those chances.
* So, according to our model, Harry Kane outperformed by 1 goal by scoring 25, ending the season with an XG of 24.

So is our model better that the one used by other people? Maybe Yes, Maybe not. As mentioned before, XG stats returns the **probabilty** of a goal being scored as a continuous variable between 0 to 1 from a variety of information that is not even included in this dataset. Our model just predicts if the series of events that were built up to the shot on goal were an indicator of a goal or not, it just predicts  a goal or no-goal. Although, it does do a pretty good job at predicting the total number of goals scored by a player in a given season. Check it out by running the code for several players like Ronaldo, Luiz Suarez etc and compare them on infogol.

* Lets inspect the events that were predicted as goals but were not actually. By inspecting the text of the events, we may be able to determine why the model choose to predict a goal for the event.

In [ ]:
# applying the index values to the events dataset so as to extract the text of those events
events.iloc[lets_test.is_goal.index][y_test==1][events.is_goal==0].text

* The output of the above code gives us exactly those 6 events which were predicted as a goal but actually were not.
* Upon inspecting the text, they should have been scored according to the placement and location of the shot. Most of those shots are from the center of the goal and more importantly, inside the box. The following plot shows the distribution of the goals and the location and placement of the shot. Majority of the goals occur when the shots are placed in the bottom corners and the center of the goal, and according to the location, majority of the goals are scored from the center of the box. Hence Harry Kane should have converted these chances bubt failed to score.

In [ ]:
fig,ax=plt.subplots(1,2,figsize=(14,6))
sns.violinplot(data=events,x='is_goal',y='shot_place',ax=ax[0])
sns.violinplot(data=events,x='is_goal',y='location',ax=ax[1])
plt.yticks(ticks=range(1,20))
plt.show()

In [ ]:
# applying the index values to the events dataset so as to extract the text of those events
predicted_as_0=events.iloc[lets_test.is_goal.index][y_test==0]
predicted_as_0[predicted_as_0.is_goal==1].text

* The code above gives us all those events that were predicted as 'no_goal' but were actually scored by Kane.
* Upon inspecting the text, the text for index 797096  says that the shot was from a difficult angle, if the above location vs is_goal plot is referred, we can see that the amount of goals scored from outside the box is very less, hence the model predicted as 'no goal'. Also, the share of goals scored from outside the box is also very less compared to the other locations which explains index 725400.
* All the other shots seem to be scorable chances which were predicted as 'no goal' by the model. Lets inspect these events further. Most of the shots seem to move diagonally or across the goal, for eg.,"right footed shot from the right side of the box to the bottom left corner". Lets check how frequently these types of goals are scored, i.e. across the goal shots where the keeper is gaurding the near post and makes it a tight angle
* As seen from the crosstab below, around a 1000 goals from the 22770 are these type of goals where its a shot across goal (where 'shot_place' is 3 or 4 and 'loaction' is 11 or 9). Hence we have explained the meaning of the False positives. The remaining events can be notched down as error in the model.


In [ ]:
pd.crosstab(events[events.is_goal==1].location,events[events.is_goal==1].shot_place)

The Logistic regression also returns the probabilities of an event being categorised. Hence we can add the probabity of each event being categorised as 1 and add them to get a probabilistic view of XG. 

In [ ]:
#to look at probabilities of scoring from an attempt on goal for Jamie Vardy
lets_test=xg_stats_final.loc[for_season[(for_season.season==2016)&(for_season.player=='jamie vardy')&(for_season.event_type==1)].index]
probabilities=logreg1.predict_proba(lets_test.drop(['is_goal','season'],axis=1))

#to get probabilities of scoring only goals or probabiltes of prediciting 1
prob=probabilities[:,1]
np.sum(prob)

If we look at the probabilities (i.e. we added the probability of an attempt being categorised as 1), we can see that Vardy should have had an XG of 25.39 goals that season

## Changing the Threshold value
But reducing the number of data points to obtain satisfactory results is not a good practice. Lets detemine different way to take care of the imbalance in the dataset
* one method is to inspect the roc (receiver operator characteristics) curve to detemine the best threshold point to obtain a resonable values for the false positive and false negative counts

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

#lets look at the data we are working with (before changing the number of data points)
x_train,x_test,y_train,y_test=train_test_split(train.drop(['is_goal','season'],axis=1),train.is_goal,test_size=0.2)
logreg.fit(x_train,y_train)
y_reg_pred=logreg.predict(x_test)

fpr,tpr,thresholds = roc_curve(y_test,logreg.predict_proba(x_test)[:,1])
plt.plot(fpr,tpr)
plt.xlabel('false positive rates')
plt.ylabel('true postitve rates')
plt.plot(fpr[max_id],tpr[max_id],marker='o') #this is the point which will maximise tpr while still having low levels of fpr (explained later)
plt.plot(fpr[238],tpr[238],marker='o',color='k')# this is the point we will use
confusion_matrix(y_test,y_reg_pred)

* Above is a plot of the true positive rates and false positive rates for different threshold values (threshold: the point after which the model would predict either event, here for example when the probablity is greater than 0.5 (50%,i.e. the threshold) the event is predicted as a goal. Our goal is to change the value of the threshold such that we can maximise the true positives and minimize the false negatives and false positives.
* The classification report below shows the recall (true positive rate), precision etc of the predicitons. Lets see what happens when we apply the optimum threshold value that is when the true positive rate is maximum and false positive rate is minimum
* This is done by detemining the geometric mean of the true positive rate and the specificity so as to obtain a good balance which represents the imbalanced dataset. Right now, our logreg model has a specificity of 98% and recall of 47.56%

In [ ]:
print(classification_report(y_test,y_reg_pred))

![](http://miro.medium.com/max/634/1*yF8hvKR9eNfqqej2JnVKzg.png)
(*Ignore the AUC value)

* The image shows the overlap of the probabilty distributions of the goal events seperated by a hue of what they actually were i.e. a goal or no goal. So when we move the threshold value either to the left or right, there will be a change in the false positives and false negatives. In our case, we want to reduce the threshold so that our false negatives reduce, but what should be our threshold value? That will be obtained by the geometric mean.
* As we will see what our graph looks like, some of the important points to look out for is the amount of overlap between the two kde plots and the peaks due to the imbalance.


In [ ]:
# lets make our own rendition of the above image
to_dist=pd.DataFrame(logreg.predict_proba(x_test)[:,1])
y_hue=y_test
y_hue.index=range(0,36876)
to_dist['hue']=y_hue
to_dist.columns=['probability','hue']
plt.figure(figsize=[10,10])
sns.distplot(to_dist[to_dist.hue==1].probability,label='prob distribution of actual events that were goals')
sns.distplot(to_dist[to_dist.hue==0].probability,label='prob distribution of actual events that were no-goals')
#line to show the current threshold value i.e. 0.5
plt.axvline(x=0.5)
plt.legend()
plt.show()

In [ ]:
#(1-fpr)=specificity
gmeans=np.sqrt(tpr*(1-fpr))
gmeans=list(gmeans)
max_id=gmeans.index(max(gmeans))
max_threshold=thresholds[max_id]
print(f'The tpr is {tpr[max_id]} and the fpr is {fpr[max_id]} for a threshold value of {thresholds[max_id]}.')

In [ ]:
#lets detemine the confusion matrix for the new threshold value
y_new_pred=[]
for i in logreg.predict_proba(x_test)[:,1]:
    if i < max_threshold:
        y_new_pred.append(0)
    else: y_new_pred.append(1)
print(confusion_matrix(y_test,y_new_pred),classification_report(y_test,y_new_pred),sep='\n')

As you can see, the False Negatives reduced drastically and incresed the recall for goal predicitons but the precision reduced also messing up the 'no-goal' predictions. lets try for increasing thresholds and compare the values

In [ ]:
y_new_pred=[]
for i in logreg.predict_proba(x_test)[:,1]:
    if i < 0.2:
        y_new_pred.append(0)
    else: y_new_pred.append(1)
print(confusion_matrix(y_test,y_new_pred),classification_report(y_test,y_new_pred),sep='\n')

In [ ]:
y_new_pred=[]
for i in logreg.predict_proba(x_test)[:,1]:
    if i < 0.3:
        y_new_pred.append(0)
    else: y_new_pred.append(1)
print(confusion_matrix(y_test,y_new_pred),classification_report(y_test,y_new_pred),sep='\n')

In [ ]:
# threshold of 0.3 seems resonable as the number of False positives have reduced 
#lets try this threshold on the players
lets_test=xg_stats_final.loc[for_season[(for_season.season==2016)&(for_season.player=='jamie vardy')&(for_season.event_type==1)].index]
y_prob_pred=logreg.predict_proba(lets_test.drop(['is_goal','season'],axis=1))

y_30_pred=[]
#30 because we have set threshold at 30%
for i in y_prob_pred[:,1]:
    if i < 0.3:
        y_30_pred.append(0)
    else: y_30_pred.append(1)
print('   0    1',f'0 {confusion_matrix(lets_test.is_goal,y_30_pred)[0]}',f'1 {confusion_matrix(lets_test.is_goal,y_30_pred)[1]}',sep='\n')


*Well would you look at that! the output is exactly the same as what we got when we reduced the sample points of 'no-goal' events and made a new model 'logreg1'.*
 *Hence, just by changing the threshold, we were able to obtain good outputs at the same time, it was achived without dropping thousands of data points, while having a true positive rate of 66.66% and a false positive rate of 6.8%.